In [1]:
import numpy as np
import cv2
import os

<h1 style="text-align: center;">
    Camera Calibration with OpenCV
</h1>

In [2]:
chessboard_size = (8,6) # Your chessboard size

# iterative termination criteria, maximum iterationm and epsilon
term_criteria = (cv2.TermCriteria_EPS+ cv2.TermCriteria_MAX_ITER, 30, 0.001)

In [3]:
# List to store object points and images point from all the images.
obj_points = list() # 3D points in heterogeneous Xi
img_points = list() # 2D points on image xi

# Defining the world coordinates for 3D points Xi
objp = np.zeros((1, chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
prev_img_shape = None

In [4]:
image_path = os.getcwd()+'/images/monocular/calibrationdata/'
images = list()
image_size = None
print(image_path)

# Preparing calibration files
for filename in os.listdir(image_path) :
    print(filename)
    img = cv2.imread(image_path+'{}'.format(filename))
    images.append(img)

d:\KMUTNB\IEE\Image Processing\010723305-Computer-Vision/images/monocular/calibrationdata/
WIN_20211024_15_44_26_Pro.jpg
WIN_20211024_15_44_28_Pro.jpg
WIN_20211024_15_44_30_Pro.jpg
WIN_20211024_15_44_32_Pro.jpg
WIN_20211024_15_44_35_Pro.jpg
WIN_20211024_15_44_37_Pro.jpg
WIN_20211024_15_44_39_Pro.jpg
WIN_20211024_15_44_41_Pro.jpg
WIN_20211024_15_44_43_Pro.jpg
WIN_20211024_15_44_45_Pro.jpg
WIN_20211024_15_44_48_Pro.jpg
WIN_20211024_15_44_50_Pro.jpg
WIN_20211024_15_44_52_Pro.jpg
WIN_20211024_15_44_54_Pro.jpg
WIN_20211024_15_44_56_Pro.jpg
WIN_20211024_15_44_58_Pro.jpg
WIN_20211024_15_45_00_Pro.jpg
WIN_20211024_15_45_03_Pro.jpg
WIN_20211024_15_45_05_Pro.jpg
WIN_20211024_15_45_07_Pro.jpg
WIN_20211024_15_45_09_Pro.jpg
WIN_20211024_15_45_11_Pro.jpg
WIN_20211024_15_45_13_Pro.jpg
WIN_20211024_15_45_15_Pro.jpg
WIN_20211024_15_45_18_Pro.jpg
WIN_20211024_15_45_20_Pro.jpg
WIN_20211024_15_45_22_Pro.jpg
WIN_20211024_15_45_24_Pro.jpg
WIN_20211024_15_45_26_Pro.jpg
WIN_20211024_15_45_28_Pro.jpg
WIN_20211

<h2 style="text-align: center">
    <a href = "https://docs.opencv.org/4.5.3/d9/d0c/group__calib3d.html#ga93efa9b0aa890de240ca32b11253dd4a"> findChessboardCorners </a>
    is a built in function for detecting the chessboard corners <br>
    <a href = "https://docs.opencv.org/4.5.3/dd/d1a/group__imgproc__feature.html#ga354e0d7c86d0d9da75de9b9701a9a87e"> cornerSubPix </a>
    is a function that uses to refine the location of the corners <br>
    <a href = "https://docs.opencv.org/4.5.3/d9/d0c/group__calib3d.html#ga6a10b0bb120c4907e5eabbcd22319022"> drawChessboardCorners </a>
    is a utility function for draw the detected corner to an image
</h2>

In [5]:
for image in images :
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(img_gray, chessboard_size, None) # xi in pixel (20,15)
    image_size = img_gray.shape
    # If found add obj points, image points afterthat refining them
    if ret == True :
        #print('Found')
        obj_points.append(objp) # Add Xi 3D
        
        corners2 = cv2.cornerSubPix(img_gray, corners, (11,11), (-1,-1), term_criteria) # Refining xi -> xi in subpixel, xi -> floating point (19.7, 15.1)
        img_points.append(corners2) # Add xi 2D

        #Draw and display the chessboard corners
        img = cv2.drawChessboardCorners(image, chessboard_size, corners2, ret)
        cv2.imshow('frame', img)
        cv2.waitKey(100)

cv2.destroyAllWindows()

<h2 style="text-align: center">
    <a href = "https://docs.opencv.org/4.5.3/d9/d0c/group__calib3d.html#ga3207604e4b1a1758aa66acb6ed5aa65d"> calibrateCamera </a>
    is the main function which you can use to calibrate a pinhole camera model <br>
</h2>

In [6]:
# Start calibration argmin ||xi - PXi||^2 , xi = PXi
print('Calibration')
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points,img_points, image_size[::-1] , None, None)

Calibration


<h2 style="text-align: center">
    <a href = "https://docs.opencv.org/4.5.3/d9/d0c/group__calib3d.html#ga7a6c4e032c97f03ba747966e6ad862b1"> getOptimalNewCameraMatrix </a>
    The function computes and returns the optimal new camera intrinsic matrix based on the free scaling parameter <br>
</h2>

In [7]:
img = images[0].copy()
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
print(mtx)
print('Optimal K')
print(newcameramtx, roi)

[[1.29968396e+03 0.00000000e+00 6.32814378e+02]
 [0.00000000e+00 1.29933998e+03 4.74721903e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Optimal K
[[1.08458081e+03 0.00000000e+00 6.26112935e+02]
 [0.00000000e+00 1.09528772e+03 4.84140676e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] (41, 59, 1192, 635)


<h2 style="text-align: center">
    <a href = "https://docs.opencv.org/4.5.3/d9/d0c/group__calib3d.html#ga69f2545a8b62a6b0fc2ee060dc30559d"> undistort </a>
    The function transforms an image to compensate radial and tangential lens distortion. <br>
</h2>

In [8]:
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
undistored = cv2.undistort(img, mtx, dist, None, None)
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]

In [9]:
mean_error = 0
for i in range(len(obj_points)) :
    reprojected_point,_ = cv2.projectPoints(obj_points[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(img_points[i], reprojected_point, cv2.NORM_L2)/len(reprojected_point)
    mean_error += error

print("Reprojection error: {}".format(mean_error))

while True :
    cv2.imshow('calibrated', dst)
    cv2.imshow('undistort', undistored)
    if cv2.waitKey(1) & 0xFF == 27 : 
        break
cv2.destroyAllWindows()

Reprojection error: 5.6861736867904975


In [13]:
#Saving parameters into numpy format
np.save("./new-camera_params/ret", ret)
np.save("./new-camera_params/K", mtx)
np.save("./new-camera_params/dist", dist)
np.save("./new-camera_params/rvecs", rvecs)
np.save("./new-camera_params/tvecs", tvecs)